# 1H Strategy Tester

Notebook ini menggabungkan seluruh pipeline backtest untuk timeframe 1 jam. Gunakan sel-sel di bawah untuk memuat data, menjalankan strategi, dan meninjau performa trade-by-trade.

## Alur kerja
1. Konfigurasikan parameter backtest melalui `StrategyProperties`.
2. Muat data 1 jam (atau gunakan fallback yang disediakan bila belum tersedia).
3. Bangun sinyal menggunakan modul `src/`.
4. Jalankan `run_backtest` dan hitung metrik performa (lokal maupun QF-Lib).
5. Bentuk `trade_log` untuk melihat entry, exit, P&L, dan statistik pendukung evaluasi.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

from backtest import performance_metrics, run_backtest
from data_loader import load_ohlcv_csv
from properties import StrategyProperties
from qflib_metrics import qflib_metrics_from_returns
from strategy import ema_vs_price_signals
from trade_analysis import failed_entries, generate_trade_log, summarise_trades


In [ ]:
plt.style.use('seaborn-v0_8')
DATA_DIR = Path('../data')
DATA_PATH = DATA_DIR / 'OKX_BTCUSDT_1H.csv'
FALLBACK_PATH = DATA_DIR / 'OKX_BTCUSDT, 1D.csv'

if DATA_PATH.exists():
    df = load_ohlcv_csv(str(DATA_PATH))
else:
    df_daily = load_ohlcv_csv(str(FALLBACK_PATH))
    df = df_daily.resample('1H').ffill()
    print('⚠️ Data 1H khusus tidak ditemukan, menggunakan resample dari data harian untuk contoh.')

print(df.head())
print(df.index.min(), '→', df.index.max())


In [ ]:
props = StrategyProperties(
    symbol='BTCUSDT',
    timeframe='1H',
    initial_capital=10_000.0,
    trading_cost_bps=5.0,
    stop_loss_pct=None,
)

positions = ema_vs_price_signals(df)
backtest = run_backtest(df, positions, **props.to_kwargs())
backtest.head()


In [ ]:
metrics = performance_metrics(backtest['equity_curve'])
qf_metrics = qflib_metrics_from_returns(backtest['strategy_ret'])
metrics, qf_metrics


In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
backtest['equity_curve'].plot(ax=ax, title='Equity Curve (1H)')
ax.set_ylabel('Equity (USD)')
fig.tight_layout()
fig


In [ ]:
trade_log = generate_trade_log(backtest, initial_capital=props.initial_capital)
trade_log.head()


In [ ]:
summary = summarise_trades(trade_log)
pd.DataFrame([summary.__dict__])


In [ ]:
entry_review = failed_entries(trade_log, threshold=0.0)
entry_review.head()


## Catatan evaluasi
Gunakan tabel `entry_review` untuk menandai trade yang gagal (net P&L ≤ 0). Anda dapat
menambahkan analisis lanjutan—misalnya memeriksa konteks indikator tambahan—dengan
menggabungkan `trade_log` dan data harga pada rentang tanggal yang sama.